<a href="https://colab.research.google.com/github/mleyvaz/AV/blob/master/Kaggle_imagenes_SVM_KEras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os 
os.environ['KAGGLE_CONFIG_DIR']="/content/gdrive/MyDrive/cancer"


In [ ]:
%cd /content/gdrive/MyDrive/cancer

In [ ]:
!kaggle datasets download -d simjeg/lymphoma-subtype-classification-fl-vs-cll

In [ ]:
!ls

In [ ]:
!apt install unzip

In [ ]:
!unzip lymphoma-subtype-classification-fl-vs-cll.zip

In [ ]:
import numpy as np
X = np.load('X.npy') # images
Y = np.load('Y.npy') # labels associated to images (0 = no IDC, 1 = IDC)

In [ ]:
def describeData(a,b):
    print('Total number of images: {}'.format(len(a)))
    print('Number of IDC(-) Images: {}'.format(np.sum(b==0)))
    print('Number of IDC(+) Images: {}'.format(np.sum(b==1)))
    print('Percentage of positive images: {:.2f}%'.format(100*np.mean(b)))
    print('Image shape (Width, Height, Channels): {}'.format(a[0].shape))
describeData(X,Y)

In [ ]:
import cv2  
from google.colab.patches import cv2_imshow # for image display
from PIL import Image 
import matplotlib.pylab as plt
imgs0 = X[Y==0] # (0 = no IDC, 1 = IDC)
imgs1 = X[Y==1] 
cv2_imshow(imgs0[100])   

In [ ]:
cv2_imshow(imgs1[100])

In [ ]:
import sklearn.model_selection as model_selection
X_train, X_test, Y_train, Y_test =  model_selection.train_test_split(X, Y, test_size=0.2)

In [ ]:
X_trainShape = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]
X_testShape = X_test.shape[1]*X_test.shape[2]*X_test.shape[3]

X_trainFlat = X_train.reshape(X_train.shape[0], X_trainShape)
X_testFlat = X_test.reshape(X_test.shape[0], X_testShape)

In [ ]:
from sklearn.svm import SVC

def svm(a,b,c,d):
    """Run LogisticRegression w/ Kfold CV"""
    model = SVC()
    model.fit(a,b)
    kfold = model_selection.KFold(n_splits=10)
    accuracy = model_selection.cross_val_score(model, c,d, cv=kfold, scoring='accuracy')
    mean = accuracy.mean() 
    stdev = accuracy.std()
    print('SVM - Training set accuracy: %s (%s)' % (mean, stdev))
    print('')
svm(X_trainFlat, Y_train, X_testFlat, Y_test)

In [ ]:
!pip install autokeras

In [ ]:
import autokeras as ak

clf = ak.ImageClassifier(max_trials = 10)

clf.fit(X_train, Y_train,epochs=2)
results = clf.predict(X_test)

In [ ]:
from keras.utils.vis_utils import plot_model
model=clf.export_model()
plot_model(model)